Create a bed-like file with regions to exclude from the variable alignment:

 drug-resistance loci, insertion sequences, repetitive gene families

 Output: a bed-like file Chromosome, start, end, gene, locus_tag


In [2]:
# Load names of genes involved in DR

dr_genes = []

with open('WHO-UCN-TB-2023.7-eng.gene_names.txt') as f:
    dr_genes = [x.strip() for x in f.readlines()]
    
dr_genes

['PPE35',
 'Rv0010c',
 'Rv0565c',
 'Rv0678',
 'Rv1129c',
 'Rv1258c',
 'Rv1979c',
 'Rv2477c',
 'Rv2680',
 'Rv2681',
 'Rv2752c',
 'Rv2983',
 'Rv3083',
 'Rv3236c',
 'aftB',
 'ahpC',
 'atpE',
 'bacA',
 'ccsA',
 'clpC1',
 'ddn',
 'dnaA',
 'eis',
 'embA',
 'embB',
 'embC',
 'embR',
 'ethA',
 'ethR',
 'fbiA',
 'fbiB',
 'fbiC',
 'fgd1',
 'gid',
 'glpK',
 'gyrA',
 'gyrB',
 'hadA',
 'inhA',
 'katG',
 'lpqB',
 'mmpL5',
 'mmpS5',
 'mshA',
 'mtrA',
 'mtrB',
 'ndh',
 'nusG',
 'panD',
 'pepQ',
 'pncA',
 'rplC',
 'rpoA',
 'rpoB',
 'rpoC',
 'rpsA',
 'rpsL',
 'rrl',
 'rrs',
 'sigE',
 'tlyA',
 'tsnR',
 'ubiA',
 'whiB6',
 'whiB7']

In [33]:
import re

pattern = r"Name=([^;]+);.*?locus_tag=([^;]+)"
pattern_mge = r"Note=([^-%]+)"

chromosome = 'MTB_anc'

annot = []
exclude = []

with open('H37Rv.RefSeq_annot.gff') as f:
    for line in f:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        bedline = []
        
        if fields[2] == 'gene':
            
            match = re.search(pattern, fields[-1])
            
            if match:
                
                name = match.group(1)
                locus_tag = match.group(2)
                start = fields[3]
                end = fields[4]
                strand = fields[6]
                
                bedline = [chromosome, start, end, strand, name, locus_tag]
                
                for fam in ['esx', 'PPE', 'PE']:
                    if name.startswith(fam):
                        exclude.append(bedline)

                if name in dr_genes or locus_tag in dr_genes:
                    exclude.append(bedline)
                    
        elif fields[2] == 'mobile_genetic_element':
            match = re.search(pattern_mge, fields[-1])
            if match:
                is_name = match.group(1)
                if is_name.startswith('IS6110'):
                    bedline = [chromosome, start, end, strand, is_name, 'NA']
                    exclude.append(bedline)
                
        elif fields[2] == 'repeat_region':
            bedline = [chromosome, start, end, strand, 'repeat_region', 'NA']
            exclude.append(bedline)
        

In [34]:
exclude


[['MTB_anc', '1', '1524', '+', 'dnaA', 'Rv0001'],
 ['MTB_anc', '5240', '7267', '+', 'gyrB', 'Rv0005'],
 ['MTB_anc', '7302', '9818', '+', 'gyrA', 'Rv0006'],
 ['MTB_anc', '13133', '13558', '-', 'Rv0010c', 'Rv0010c'],
 ['MTB_anc', '21637', '23181', '-', 'repeat_region', 'NA'],
 ['MTB_anc', '79486', '80193', '+', 'repeat_region', 'NA'],
 ['MTB_anc', '79486', '80193', '+', 'repeat_region', 'NA'],
 ['MTB_anc', '103710', '104663', '-', 'repeat_region', 'NA'],
 ['MTB_anc', '105324', '106715', '+', 'PPE1', 'Rv0096'],
 ['MTB_anc', '131382', '132872', '+', 'PE_PGRS1', 'Rv0109'],
 ['MTB_anc', '149533', '150996', '+', 'PE_PGRS2', 'Rv0124'],
 ['MTB_anc', '152324', '154129', '+', 'repeat_region', 'NA'],
 ['MTB_anc', '152324', '154129', '+', 'repeat_region', 'NA'],
 ['MTB_anc', '152324', '154129', '+', 'repeat_region', 'NA'],
 ['MTB_anc', '177543', '179309', '-', 'PE1', 'Rv0151c'],
 ['MTB_anc', '179319', '180896', '-', 'PE2', 'Rv0152c'],
 ['MTB_anc', '187433', '188839', '-', 'PE3', 'Rv0159c'],
 ['MTB_

# Write to file

In [35]:
n_excluded = set()

with open('excluded_regions.bed', 'w') as f:
    for bedline in exclude:
        start = int(bedline[1])
        end = int(bedline[2])
        for i in range(start, end):
            n_excluded.add(i)
            
        f.write('\t'.join(bedline) + '\n')
        
print(f'Number of excluded positions: {len(n_excluded)}')

Number of excluded positions: 466849
